In [2]:
import pandas as pd
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from transformers import TrainingArguments, Trainer

from transformers import EvalPrediction
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import torch
torch.cuda.is_available()

from torch.utils.data.dataset import Dataset
from torch.utils.data.sampler import *

import pickle
from tqdm.notebook import tqdm
tqdm.pandas()

import random
import os
import numpy as np

import re

In [4]:
raw_data = pd.read_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\관세용어사전_셔플.xlsx")
raw_data

,단어이름,단어 설명
0,인터넷 네트워크 정보센터,인터넷 관련 자원과 정보의 관리를 행하는 기관. 각국에 네트워크 정보 센터(NIC)...
1,현실전손,"절대전손(Absolute Total Loss)이라고도 하며, 화물이 현실로 전손된 ..."
2,국경 내 장벽,"관세나 비관세장벽과 같은 국경 간 장벽은 아니나, 1차적으로 국내경제를 위해 채택하..."
3,내항능력,"선박이 그 운항을 마치기 위하여 선체, 기관, 속구를 완비하고 필요한 승무원을 배치..."
4,간이 입출항절차,외국무역선(기)이 개항에 입항하여 물품을 하역하지 않고 입항한 때로부터 24시간이 ...
...,...,...
4884,대사관물품,외교공관에 쓰여지는 각종 비품 및 설비등으로 관세법상 면책의 특권이 인정되는 물품.
4885,견품반출,"보세구역에 장치된 외국물품을 검사, 확인용 등의 목적으로 일시보세구역 밖으로 반출하..."
4886,일괄보험료율,보험에 있어 보험대상인 재산이 여럿인 경우 선박 전체를 통틀어 적용되는 하나의 보험료율.
4887,우담수손,해수손(海水損; sea water damage)에 대비되는 말로서 염분이 섞여 있지...


In [5]:
def cleasing(text):
    repl =''
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 자음, 모음 제거
    text = re.sub(pattern= pattern, repl=repl, string=text)
    #pattern = '[^\w\s]' # 특수기호 제거
    pattern = '[^가-히\s]' # 특수기호 제거
    text = re.sub(pattern= pattern, repl=repl, string=text)
    pattern = '<[^>]*>' # html 제거
    text = re.sub(pattern = pattern, repl='',string=text)
    
    pattern = ' +'
    text = re.sub(pattern = pattern, repl=' ', string=text)
    
    pattern = '"'
    text = re.sub(pattern = pattern, repl='', string=text)
    
    pattern = "'"
    text = re.sub(pattern=pattern, repl='', string=text)
    
    pattern = ","
    text = re.sub(pattern=pattern, repl='', string=text)
    
    text = text.strip()
    
    return text

In [6]:
raw_data['단어이름'] = raw_data['단어이름'].astype(str)
raw_data['단어이름'] = raw_data['단어이름'].apply(lambda x:cleasing(x))

In [7]:
raw_data['단어 설명'] = raw_data['단어 설명'].astype(str)
raw_data['단어 설명'] = raw_data['단어 설명'].apply(lambda x:cleasing(x))


In [8]:
raw_data


,단어이름,단어 설명
0,인터넷 네트워크 정보센터,인터넷 관련 자원과 정보의 관리를 행하는 기관 각국에 네트워크 정보 센터가 있으며 ...
1,현실전손,절대전손 이라고도 하며 화물이 현실로 전손된 경우 또는 그 점유가 박탈되어 회복할 ...
2,국경 내 장벽,관세나 비관세장벽과 같은 국경 간 장벽은 아니나 차적으로 국내경제를 위해 채택하지만...
3,내항능력,선박이 그 운항을 마치기 위하여 선체 기관 속구를 완비하고 필요한 승무원을 배치하고...
4,간이 입출항절차,외국무역선기이 개항에 입항하여 물품을 하역하지 않고 입항한 때로부터 시간이 내출항하...
...,...,...
4884,대사관물품,외교공관에 쓰여지는 각종 비품 및 설비등으로 관세법상 면책의 특권이 인정되는 물품
4885,견품반출,보세구역에 장치된 외국물품을 검사 확인용 등의 목적으로 일시보세구역 밖으로 반출하는 행위
4886,일괄보험료율,보험에 있어 보험대상인 재산이 여럿인 경우 선박 전체를 통틀어 적용되는 하나의 보험료율
4887,우담수손,해수손 에 대비되는 말로서 염분이 섞여 있지 않은 담수손을 말함 화물이 운송중 장치...


In [13]:
raw_data.to_excel(r"C:\Users\user\Desktop\학술대회 2차\데이터\통상용어사전_전처리.xlsx")